<a href="https://colab.research.google.com/github/agkhairnar/ECON-626-PS/blob/master/topic_01_bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

It is critically important to mimic the original experiment in every way, down to the last detail, in order to compute the bootstrapped standard errors correctly. In this short example, we demonstrate how a small subtlety leads to different standard errors.

Suppose we run an experiment, and have the following data. 

In [ ]:
import numpy as np
np.random.seed(100)

y = np.array([4,3,2,4,1,2,3,5,2,4,7,3,5,6,4,3,1,3,4,3])
t = np.array([0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1])
effect = y[np.where(t == 1)[0]].mean() - y[np.where(t == 0)[0]].mean()

print("Estimated effect of treatment is: " + str(round(effect, 3)))

print("There are " + str(sum(t)) + " treated units and " + 
      str(len(t) - sum(t)) + " control units")

Estimated effect of treatment is: 1.02
There are 11 treated units and 9 control units


Before we move on, take a moment to think about the different ways we might compute the bootstrapped standard errors.

Be specific about the re-sampling design that you might implement, and why you made the design choices that you did.

In doing so, notice that there are nine control units, and eleven treated units. There are a few plausible hypotheses:


1.   There are supposed to be exactly nine control units and exactly eleven treated units.
2.   Each unit is allocated with 55% probability to treatment.
2.   Each unit is allocated with 50% probability to treatment. Due to some random chance, there are more treated than control units.

Notice how the setup of the bootstrap computation differs for each one, and how the results change.



In [ ]:
samples1 = []
samples2 = []
samples3 = []

# Pick a large number of iterations to wash away sampling noise
for i in range(50000):

  # Approach 1: exactly nine and eleven units
  # So we sample the nine control units and eleven treated units separately
  treatment_mean = y[np.random.choice(np.where(t == 1)[0], 
    len(np.where(t == 1)[0]), replace = True)].mean()
  control_mean = y[np.random.choice(np.where(t == 0)[0], 
    len(np.where(t == 0)[0]), replace = True)].mean()
  samples1.append(treatment_mean - control_mean)

  # Approach 2: units get allocated to treatment with 55% chance
  # So we sample among all twenty units with equal weights, 
  # and so our samples will have 55% treatment on average
  index = np.random.choice(range(len(t)), len(t), replace = True)
  if len(np.where(t[index] == 1)[0]) >0 and len(np.where(t[index] == 0)[0]) >0:
    treatment_mean = y[index][np.where(t[index] == 1)[0]].mean()
    control_mean = y[index][np.where(t[index] == 0)[0]].mean()
    samples2.append(treatment_mean - control_mean)

  # Approach 3: units get allocated to treatment with 50% chance
  # So we sample among all twenty units with unequal weights, 
  # and so our samples will have 50% treatment on average
  p = np.repeat(sum(t)/len(t), len(np.where(t == 0)[0]))
  p = np.append(p, np.repeat(1 - sum(t)/len(t), len(np.where(t == 1)[0])))
  index = np.random.choice(range(len(t)), len(t), replace = True, p = p/sum(p))
  if len(np.where(t[index] == 1)[0]) >0 and len(np.where(t[index] == 0)[0]) >0:
    treatment_mean = y[index][np.where(t[index] == 1)[0]].mean()
    control_mean = y[index][np.where(t[index] == 0)[0]].mean()
    samples3.append(treatment_mean - control_mean)

print(round(np.std(samples1), 4))
print(round(np.std(samples2), 4))
print(round(np.std(samples3), 4))

0.6171
0.6332
0.6408
